In [ ]:
import tushare as ts
import matplotlib.pyplot as plt
import pandas as pd
import pymysql.cursors
import time
import pymysql
import numpy as np


ts.set_token('7f9f65ad332e85a7f279e09f06067851cbf808a79a56da7cb5659de8')#设置token，只需设置一次
api = ts.pro_api()#初始化接口

#获取A股的K线数据，带有复权参数“adj”、均线参数“ma”、数据频度参数“freq”
data = ts.pro_bar(pro_api=api, ts_code='000009.SZ', adj='qfq', start_date='20170101', end_date='20181011',ma=5,freq='D')

#1、如何下载一只股票的全部数据？
#答：直接调用pro_bar接口，将时间设置为上市日期到某天，一次性向服务器请求数据的方式是行不通的。理由很简单，人家服务器不是无限量的，咱不能搞太狠。正确姿势这里提供一个正确的例子。

def fetchklinedata(code):
    filename = 'your path'
    if not os.path.exists(filename):
        end_date = datetime.strftime(datetime.now(), '%Y%m%d')#获取当前时间
        outputflag = True
        api = tushare.pro_api()
        while outputflag:#循环判断，直到返还的数据为空
           data=tushare.pro_bar(pro_api=api,ts_code=code,
                                end_date=end_date,asset='E',adj=None,freq='D')#请求本次数据
            if data.empty == True:
                outputflag = False
            else:
                #计算下次请求数据的截止日期
                next_end_date = datetime.strptime(data.iloc[-1]['trade_date'],
                                                  '%Y%m%d') - timedelta(hours=24)
                end_date = datetime.strftime(next_end_date, '%Y%m%d')
                #写csv文件
                if os.path.exists(filename):
                    data.to_csv(filename, header=None, mode='a')#追加写入模式
                else:
                    data.to_csv(filename)
#以股票的日线数据为例，来计算周线、月线是非常方便的。这里需要使用pandas中的resample这个函数。
data_day = ts.pro_bar(pro_api=api, ts_code='000009.SZ', adj='qfq', start_date='20170101', end_date='20181011',ma=5,freq='D')
data_day['trade_date'] = pandas.to_datetime(data_day['trade_date'])
data_week = data_day.resample(rule='W', on='trade_date', label='left', closed='left').agg(
        {'open': 'first',
         'high': 'max',
         'low': 'min',
         'close': 'last',
         'vol': 'sum',
         })
#具体的时间周期字段rule
"""
rule:
    B       business day frequency
    C       custom business day frequency (experimental)
    D       calendar day frequency
    W       weekly frequency
    M       month end frequency
    SM      semi-month end frequency (15th and end of month)
    BM      business month end frequency
    CBM     custom business month end frequency
    MS      month start frequency
    SMS     semi-month start frequency (1st and 15th)
    BMS     business month start frequency
    CBMS    custom business month start frequency
    Q       quarter end frequency
    BQ      business quarter endfrequency
    QS      quarter start frequency
    BQS     business quarter start frequency
    A       year end frequency
    BA      business year end frequency
    AS      year start frequency
    BAS     business year start frequency
    BH      business hour frequency
    H       hourly frequency
    T       minutely frequency
    S       secondly frequency
    L       milliseonds
    U       microseconds
    N       nanoseconds

"""
#下载一个上市公司完整的财务指标
def fetchfina_indicator(code):
    filename = 'your path'
    if not os.path.exists(filename):
        end_date = datetime.strftime(datetime.now(), '%Y%m%d')
        outputflag = True
        api = tushare.pro_api()
        while outputflag:
            data = api.fina_indicator(ts_code=code, end_date=end_date)
            if data.empty == True:
                outputflag = False
            else:
                next_end_date = datetime.strptime(
                    data.iloc[-1]['end_date'], '%Y%m%d') - timedelta(hours=24)
                end_date = datetime.strftime(next_end_date, '%Y%m%d')
                if os.path.exists(filename):
                    data.to_csv(filename, header=None, mode='a')
                else:
                    data.to_csv(filename)
                    
#tushare pro提供了市场上所有指数的基本信息、行情数据、指数的成分与权重以及每日指标四个接口。

#1、指数基本信息，例子中申请获取中证指数
data = api.index_basic(market='CSI')

#指数日线行情

data = api.index_daily(ts_code='000843.CSI', start_date='20180101', end_date='20181122')

#4、大盘指数每日指标

data = api.index_dailybasic(trade_date='20181122', fields='ts_code,trade_date,turnover_rate,pe,pe_ttm,pb')
